In [1]:
import h5py
from Bio import SeqIO, AlignIO
import io
import numpy as np
import os

pfam_path = '/home/cactuskid13/struct_data/pfam/Pfam-A.full'
import gzip

def yield_alns(pfam_path , verbose = False):    
    with open(pfam_path, 'r', encoding='ISO-8859-1') as f:
        aln= ''
        acc = None
        count =0
        lCount = 0
        for i,l in enumerate( f ):
            lCount += 1
            if verbose == True:
                if i > 3000 and i < 4000:
                    print(l)
            
            if lCount < 10**6:
                aln+=l
            
            if acc is None and 'AC' in l:
                acc = l.split()[2] 
            
            if l == '//\n':
                if lCount > 10**6:
                    print(acc + 'truncated')
                if count < 8063 or count > 8065:
                    msa = AlignIO.read(io.StringIO(aln), "stockholm")
                else:
                    print('skipping')
                    msa = None
                idPfam = acc
                acc = None
                aln = ''
                lCount = 0
                yield idPfam, msa
                if count % 1000 == 0 :
                    print(msa)
                count+=1

In [22]:
if os.path.isfile('Pfam-A.full.h5'):
    os.remove('Pfam-A.full.h5')
open('Pfam-A.full.h5', 'a').close()

with h5py.File('Pfam-A.full' +'.h5', 'r+') as hf:
    i = 0
    for pfam_id, msa in yield_alns(pfam_path):
        i += 1
        print(i)
        if not hf.get(pfam_id):
            try:
                align_list = list()
                for rec in msa:
                    align_list.append(np.array(list(rec.upper()), np.character))
                try:
                    align_array = np.array(align_list)
                    hf.create_dataset(pfam_id,  data=align_array)
                except:
                    print('exception')
            except:
                print('for loop exception')
        else:
            print(pfam_id)

1


/home/osboxes/miniconda3/envs/structML/lib/python3.7/site-packages/ipykernel_launcher.py:16: DeprecationWarning: Converting `np.character` to a dtype is deprecated. The current result is `np.dtype(np.str_)` which is not strictly correct. Note that `np.character` is generally deprecated and 'S1' should be used.
  app.launch_new_instance()


here  [[b'-' b'-' b'-' ... b'-' b'-' b'-']
 [b'-' b'-' b'-' ... b'-' b'-' b'-']
 [b'-' b'-' b'-' ... b'-' b'-' b'-']
 ...
 [b'-' b'-' b'-' ... b'-' b'-' b'-']
 [b'-' b'-' b'-' ... b'-' b'-' b'-']
 [b'-' b'-' b'-' ... b'-' b'-' b'-']] (9587, 120)
Alignment with 9587 rows and 120 columns
------------ALQFHEEH--GE--VCPAQW--HK--------...--- A0A1I4YJU4_9ENTR/160-195
------------AFQFTDKH--GE--VCPAGW--KP--------...--- F6R8L2_ORNAN/163-198
------------ALQYVASH-pGE--VCPAKW--KE--------...--- A0A2S6MVG1_9RHIZ/154-186
------------ALQRSADG--QA--YTPADW--QP--------...--- A0A2A2JWD4_9BILA/164-196
-----------a-AQYVAAH-pGE--VCPAKW--TE--------...--- W0UZH3_9BURK/154-186
------------AFQYVREH-aGE--ACPAGW--TP--------...--- F8A8Q3_THEID/158-196
---------ayt---HVQVK--GE--VCPANW--EE--------...--- A0A0A2M5Q4_9FLAO/176-211
------------AFQFVETH--GE--VCPASW--TP--------...--- A0A3B3TJV1_9TELE/211-246
------------SLQLTDNY--SV--ATPADW--KD--------...--- K9WY66_9NOST/156-196
------------ALQFHQEH--GE--VCPAGW--NK--------.

In [ ]:
101, 104, 160, 173, 177, 181